# CNN scraping
This notebook aims to build a scaper of the news in https://www.cnnbrasil.com.br/

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta

# Path to geckodriver executable
geckodriver_path = '/snap/bin/firefox.geckodriver'


s = Service(executable_path=geckodriver_path)
# Create a Firefox webdriver instance
# opens a window
driver = webdriver.Firefox(service=s)

In [2]:
driver.get('https://www.cnnbrasil.com.br/politica/')

In [3]:
scrolls = 10

current_height = driver.execute_script("return document.body.scrollHeight")

for i in range(scrolls):
    # scroll to the end of the page
    driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
    sleep(1)
    driver.find_element(By.CSS_SELECTOR, value='.block-list-get-more-btn').click()
    
    # repeat the scroll
    driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
    current_height = driver.execute_script("return document.body.scrollHeight")
        
    print(f'Scroll {i + 1}, height={current_height}')
    
    # time for the page to load
    sleep(4)

Scroll 1, height=7486
Scroll 2, height=11018
Scroll 3, height=14526
Scroll 4, height=18034
Scroll 5, height=21518
Scroll 6, height=25026
Scroll 7, height=28462
Scroll 8, height=32018
Scroll 9, height=35502
Scroll 10, height=38938


In [ ]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'lxml')